In [ ]:
! rm -rf sample_data
! cp drive/MyDrive/DDEQ.zip DDEQ.zip
! unzip DDEQ.zip

Archive:  DDEQ.zip
   creating: DDEQ/
  inflating: DDEQ/test_values.csv    
  inflating: DDEQ/submission_format.csv  
  inflating: DDEQ/train_labels.csv   
  inflating: DDEQ/train_values.csv   


In [ ]:
!zip models.zip -r models
!cp models.zip drive/MyDrive/models.zip

  adding: models/ (stored 0%)
  adding: models/model1.txt (deflated 70%)
  adding: models/model0.txt (deflated 69%)
  adding: models/model2.txt (deflated 70%)
  adding: models/model3.txt (deflated 69%)
  adding: models/model4.txt (deflated 70%)


In [ ]:
import pandas as pd
from tqdm import tqdm
import random
import math
import numpy as np
import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_boston

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import roc_curve,plot_roc_curve, balanced_accuracy_score

In [ ]:
tv = pd.read_csv('/content/DDEQ/train_values.csv')
tv.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
tl = pd.read_csv('/content/DDEQ/train_labels.csv')
tl.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [ ]:
tt = tv.merge(tl, how='left', on=['building_id'])
tt.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,3
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,2
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,3
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,2
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,3


In [ ]:
tt_e = tt.drop('building_id',1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
mtx = tt_e.corr()

In [ ]:
mtxc = mtx

In [ ]:
mtxc['damage_grade'] = mtxc['damage_grade'].abs()

In [ ]:
lst = mtxc[mtxc['damage_grade'] > 0.05].index

In [ ]:
lst

Index(['geo_level_1_id', 'count_floors_pre_eq', 'area_percentage',
       'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'count_families',
       'has_secondary_use', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'damage_grade'],
      dtype='object')

In [ ]:
tt_e = tt_e[lst]

In [ ]:
tt_e.head()

,geo_level_1_id,count_floors_pre_eq,area_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,count_families,has_secondary_use,has_secondary_use_hotel,has_secondary_use_rental,damage_grade
0,6,2,6,1,1,0,0,0,0,0,0,0,1,0,0,0,3
1,8,2,8,0,1,0,0,0,0,0,0,0,1,0,0,0,2
2,21,2,5,0,1,0,0,0,0,0,0,0,1,0,0,0,3
3,22,2,6,0,1,0,0,0,1,1,0,0,1,0,0,0,2
4,11,3,8,1,0,0,0,0,0,0,0,0,1,0,0,0,3


In [ ]:
for c in lst:
  tt_e[c] = tt_e[c] /tt_e[c].abs().max()

In [ ]:
Train = []
for idx, rw in tqdm(tt_e.iterrows()):
  Train.append(([rw['geo_level_1_id'],rw['geo_level_2_id'],rw['count_floors_pre_eq'],rw['area_percentage'],rw['has_superstructure_adobe_mud'],rw['has_superstructure_mud_mortar_stone'],rw['has_superstructure_stone_flag'],
                 rw['has_superstructure_cement_mortar_stone'],rw['has_superstructure_cement_mortar_brick'],rw['has_superstructure_timber'],rw['has_superstructure_bamboo'],rw['has_superstructure_rc_non_engineered'],rw['has_superstructure_rc_engineered'],
                 rw['has_secondary_use'],rw['has_secondary_use_hotel'],rw['has_secondary_use_rental'],rw['count_families']], str(int(rw['damage_grade']) - 1)))

260601it [00:35, 7347.69it/s]


In [ ]:
Train = []
for idx, rw in tqdm(tt_e.iterrows()):
  Train.append(([rw['geo_level_1_id'],rw['count_floors_pre_eq'],rw['area_percentage'],rw['has_superstructure_mud_mortar_stone'],rw['has_superstructure_cement_mortar_brick'],rw['has_superstructure_rc_non_engineered']
                 ,rw['has_superstructure_rc_engineered'],rw['has_secondary_use'],rw['has_secondary_use_rental'],rw['has_superstructure_cement_mortar_stone'],rw['has_superstructure_stone_flag'],rw['has_superstructure_timber'],
                 rw['has_superstructure_bamboo']], str(int(rw['damage_grade']) - 1)))

260601it [00:23, 10986.28it/s]


In [ ]:
X,y = [], []
for xx, yy in tqdm(Train):
  X.append(xx)
  y.append(yy)

X = np.asarray(X)
y = np.asarray(y)

100%|██████████| 260601/260601 [00:00<00:00, 1819658.62it/s]


In [ ]:
tstl = pd.read_csv('/content/DDEQ/test_values.csv',dtype={'building_id':'string'})
tstl.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,...,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,...,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
tstlst = []
for l in lst[:6]:
  tstlst.append(l)
tstlst.append('building_id')

In [ ]:
tstl = tstl[tstlst]
tstl.head()

,count_floors_pre_eq,area_percentage,has_superstructure_mud_mortar_stone,has_superstructure_cement_mortar_brick,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,building_id
0,3,7,1,0,0,0,300051
1,2,13,1,0,0,0,99355
2,2,4,1,0,0,0,890251
3,1,19,0,1,0,0,745817
4,3,8,1,0,0,0,421793


In [ ]:
Test = []
for idx, rw in tqdm(tstl.iterrows()):
  Test.append(([rw['geo_level_1_id'],rw['count_floors_pre_eq'],rw['area_percentage'],rw['has_superstructure_mud_mortar_stone'],rw['has_superstructure_cement_mortar_brick'],rw['has_superstructure_rc_non_engineered']
                 ,rw['has_superstructure_rc_engineered'],rw['has_secondary_use'],rw['has_secondary_use_rental'],rw['has_superstructure_cement_mortar_stone'],rw['has_superstructure_stone_flag']], str(int(rw['building_id']))))

86868it [00:07, 11611.33it/s]


In [ ]:
Test = []
for idx, rw in tqdm(tstl.iterrows()):
  Test.append(([rw['geo_level_1_id'],rw['geo_level_2_id'],rw['count_floors_pre_eq'],rw['area_percentage'],rw['has_superstructure_adobe_mud'],rw['has_superstructure_mud_mortar_stone'],rw['has_superstructure_stone_flag'],
                 rw['has_superstructure_cement_mortar_stone'],rw['has_superstructure_cement_mortar_brick'],rw['has_superstructure_timber'],rw['has_superstructure_bamboo'],rw['has_superstructure_rc_non_engineered'],rw['has_superstructure_rc_engineered'],
                 rw['has_secondary_use'],rw['has_secondary_use_hotel'],rw['has_secondary_use_rental'],rw['count_families']], str(int(rw['building_id']))))

86868it [00:09, 8781.99it/s]


In [ ]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score 
from sklearn.model_selection import KFold

In [ ]:
def threshold_arr(array):
    # Get major confidence-scored predicted value.
    new_arr = []
    for ix, val in enumerate(array):
        loc = np.array(val).argmax(axis=0)
        k = list(np.zeros((len(val))))
        k[loc]=1
        new_arr.append(k)
        
    return np.array(new_arr)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=2022)

In [ ]:
lgb_params = {'objective': 'multiclass','num_class':3, 'metric': 'multi_error','learning_rate': 1e-2, 'max_depth': -1, 'num_leaves':30,
         'min_sum_hessian_in_leaf':0.1, 'max_bin':8192, 'verbosity':1,'num_threads':6,'seed':1881}

In [ ]:
for ix, (train_index, test_index) in enumerate(kf.split(X)):
  x_train, x_val, y_train, y_val= X[train_index], X[test_index], y[train_index], y[test_index]
  train_data = lgb.Dataset(x_train, label=y_train)
  val_data = lgb.Dataset(x_val, label=y_val)
  lgb_clf = lgb.train(lgb_params,
                        train_data,
                        20000,
                        valid_sets = [val_data],
                        early_stopping_rounds=3000,
                        verbose_eval = 1000)
  y_pred = lgb_clf.predict(x_val)
  print("F1-MICRO SCORE: ", f1_score(np.array(pd.get_dummies(y_val)), threshold_arr(y_pred), average='micro'))
  lgb_clf.save_model(f'models/model{ix}.txt')

In [ ]:
models = []
for i in range(5):
    model = lgb.Booster(model_file=f'models/model{i}.txt')

    # y_pred = model.predict(X)
    # score  = f1_score(np.array(pd.get_dummies(y)), threshold_arr(y_pred), average='micro')
    # print("F1-MICRO SCORE: ", score)
    models.append(model)

In [ ]:
def ensemble(models, x):
    # Ensemble K-Fold CV models with adding all confidence score by class.
    y_preds = []
    
    for model in models:
        y_pred = model.predict(x)
        y_preds.append(y_pred)
        
    init_y_pred = y_preds[0]
    for ypred in y_preds[1:]:
        init_y_pred += ypred
        
    y_pred = threshold_arr(init_y_pred)
    
    return y_pred

In [ ]:
y_pred = ensemble(models, test_x)
y_pred = y_pred.argmax(axis=1)+1

In [ ]:
test_x = []
for t in Test:
  test_x.append(t[0])

In [ ]:
preds = lgb_clf.predict(test_x)

In [ ]:
prediction = []
for i in range(len(y_pred)):
  prediction.append([Test[i][1], y_pred[i]])

In [ ]:
sub = pd.DataFrame(prediction,columns=['building_id','damage_grade'])
sub.head()

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,2
4,421793,3


In [ ]:
sub.to_csv('submission.csv',index = False)

In [ ]:
smpl = pd.read_csv('/content/DDEQ/submission_format.csv')
smpl.head()

,building_id,damage_grade
0,300051,1
1,99355,1
2,890251,1
3,745817,1
4,421793,1
